<a href="https://colab.research.google.com/github/cooper-hird/WAW-score-NCAAM/blob/main/All_Game_Scores_Page.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from collections import namedtuple


######################################################################################################
            ### NCAA.com Box Scores Request ###
######################################################################################################
# Define the URL for the GET request
url = 'https://www.ncaa.com/scoreboard/basketball-men/d1/2024/12/04/all-conf'

# Send the GET request
response = requests.get(url)

games = []

############### Check for a response ###############
if 'text/html' in response.headers.get('content-type', ''):
    soup = BeautifulSoup(response.content, 'html.parser')
    #####Get Page Title#####
    #title = soup.title.string
    #print(f"Page title: {title}")

    ############### Pull All Team Names, Scores, and Determine Winner #################
    game_records = {}
    game_pods = soup.find_all('div', class_='gamePod')
    for game in game_pods:
        # Find team info (team name/score/rank)
        teams = game.find_all('span', class_='gamePod-game-team-name')
        scores = game.find_all('span', class_='gamePod-game-team-score')
        ranks = game.find_all('span', class_='gamePod-game-team-rank')

        # Ensure that there are two teams and two scores per game (home and away)
        if len(teams) == 2 and len(scores) == 2:
            # Adjust the order to print Home Team first and Away Team second
            home_team = teams[1].text.strip()  # Home team is the second in the list
            away_team = teams[0].text.strip()  # Away team is the first in the list

            home_rank = ranks[1].text.strip() or 'NR'  # Home team rank is the second rank
            away_rank = ranks[0].text.strip() or 'NR'  # Away team rank is the first rank

            home_score = int(scores[1].text.strip())  # Home team score is the second score
            away_score = int(scores[0].text.strip())  # Away team score is the first score

        # Identify winner
        winner = None
        if home_score > away_score:
            winner = home_team
        elif away_score > home_score:
            winner = away_team
        else:
            winner = "I guess this sport has ties?"

        # Print list
        #print(f"Home Team: #{home_rank} {home_team}, Away Team: #{away_rank} {away_team}, Scores: {home_score}-{away_score}, \n Winner: {winner}")
        defaultrank = 400
        default_wawscore = 0
        game = {
            'home_team_list': home_team,
            'away_team_list': away_team,
            'home_score_list': home_score,
            'away_score_list': away_score,
            'home_rank_list': defaultrank,
            'away_rank_list': defaultrank,
            'winner_list': winner,
            'composite_score': default_wawscore
              }
        games.append(game)

#    for x in range(len(home_teams)):
#        print(f"Home Team: {home_teams[x]}, Away Team: {away_teams[x]}, Scores: {home_scores[x]}-{away_scores[x]}, \n Winner: {winners[x]}")
else:
    print("The response is not in HTML format or failed to fetch. Currently, this script only works with HTML input.")



######################################################################################################
            ### NCAAM NET Rankings Request ###
######################################################################################################
url = 'https://www.ncaa.com/rankings/basketball-men/d1/ncaa-mens-basketball-net-rankings'

response = requests.get(url)
teams_rankings_dict = {}
############### Check for a response ###############
if 'text/html' in response.headers.get('content-type', ''):
    soup = BeautifulSoup(response.content, 'html.parser')
    #####Get Page Title#####
    #title = soup.title.string
    #print(f"Page title: {title}")

    rows = soup.find_all('div', class_='layout-content')
    tr_data = []
    td_data = []
    for row in rows:
      trs = row.find_all('tr')
      for tr in trs:
        tds = tr.find_all('td')
        for td in tds:
          td_data.append(td.text.strip())

    # Group the data into chunks of 12
    grouped_data = [td_data[i:i+12] for i in range(0, len(td_data), 12)]


    for i, group in enumerate(grouped_data):
        teams_rankings_dict[group[2]] = group[0]

else:
    print("The response is not in HTML format or failed to fetch. Currently, this script only works with HTML input.")


######################################################################################################
            ### Calculating WAW Score (composite game score) ###
######################################################################################################
for game in games:
    # Update the rank of the home team from the teams_rankings_dict
    game['home_rank_list'] = int(teams_rankings_dict.get(game['home_team_list'], 400))

    # Update the rank of the away team from the teams_rankings_dict
    game['away_rank_list'] = int(teams_rankings_dict.get(game['away_team_list'], 400))

    final_score_factor = 2
    ranking_factor = 1
    upset_factor = .5

    final_score_weight = 0.7
    ranking_score_weight = 0.2
    upset_score_weight = 0.1

    closeness_score = max(0, 100 - (abs(game['home_score_list'] - game['away_score_list']) * final_score_factor))
    rankings_score = max(0, min(100, 100 - (((game['home_rank_list']+game['away_rank_list'])/2) * ranking_factor)))
    upset_score = 0
    # Calculate upset score (if the winning team has a lower rank than the losing team)
    if game['winner_list'] == game['home_team_list']:
        if game['home_rank_list'] > game['away_rank_list']:  # Home team is ranked worse (higher number)
         #   upset_score = min(100, (game['away_rank_list'] - game['home_rank_list']) * upset_factor)  # Scaled difference
            upset_score = max(0, min(100, ((game['home_rank_list'] - game['away_rank_list']) * upset_factor) * (1 - (game['away_rank_list'] / 400))))  # Normalize upset based on rank

        else:
            upset_score = 0  # No upset if home team has a better rank
    elif game['winner_list'] == game['away_team_list']:
        if game['away_rank_list'] > game['home_rank_list']:  # Away team is ranked worse (higher number)
        #    upset_score = min(100, (game['home_rank_list'] - game['away_rank_list']) * upset_factor)  # Scaled difference
            upset_score = max(0, min(100, ((game['away_rank_list'] - game['home_rank_list']) * upset_factor) * (1 - (game['home_rank_list'] / 400))))  # Normalize upset based on rank
        else:
            upset_score = 0  # No upset if away team has a better rank
    else:
        upset_score = 0  # No upset in the case of a tie (though ties are unlikely in basketball)


    game['composite_score'] = round((closeness_score * final_score_weight) + (rankings_score * ranking_score_weight) + (upset_score * upset_score_weight),1)


# Sort the games in descending order based on the WAW score
sorted_games = sorted(games, key=lambda g: g['composite_score'], reverse=True)

# Print the sorted games with their WAW scores
for game in sorted_games:
    ########## CSV list ##########
    print(f"{game['home_team_list']}, {game['away_team_list']}, {game['home_score_list']}, {game['away_score_list']}, {game['home_rank_list']}, {game['away_rank_list']}, {game['winner_list']}, {game['composite_score']}, |")
    print("\n")


    ########### Fancy looking output ##########
    print(f"Home Team: {game['home_team_list']} (Rank: {game['home_rank_list']}) vs Away Team: {game['away_team_list']} (Rank: {game['away_rank_list']})")
    print(f"Scores: {game['home_score_list']} - {game['away_score_list']}")
    print(f"Winner: {game['winner_list']}")
    if game['winner_list'] == game['home_team_list'] and game['home_rank_list'] > game['away_rank_list']:
        print(f"Upset Alert")
    print(f"WAW Score: {game['composite_score']}")
    print()  # Newline for readability